In [44]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv

from xcontest.xcontest import login, flight_days
from wetter.wetter import load_raw_dataframes,get_wind_data,get_rain_data

load_dotenv()

FLUGBAHN_DIR = 30 # 03 & 21

timeframe_start,timeframe_end = 2019010100,2023123123

df_wind = get_wind_data(timeframe_start,timeframe_end , 13)
df_rain = get_rain_data(timeframe_start,timeframe_end , 13)
df_wind = df_wind.join(df_rain.set_index("MESS_DATUM"), on="MESS_DATUM", validate="1:1" )


df_wind['wind_alignment'] = df_wind.apply(lambda row: np.sin((row['direction'] - FLUGBAHN_DIR )* np.pi / 180.)**2 , axis=1)

print(df_wind.head())

driver = login()

flight_day_list= flight_days(driver, 'Metzingen')

        MESS_DATUM  strength  direction  precip  wind_alignment
604397  2019010113      10.4        290     0.0        0.969846
604421  2019010213       7.5        340     0.0        0.586824
604445  2019010313       3.9        300     0.0        1.000000
604469  2019010413       7.9        280     0.2        0.883022
604493  2019010513       6.2        320     0.0        0.883022
['10.02.2024', '09.09.2023', '11.08.2023', '05.08.2023', '07.07.2023', '18.06.2023', '10.06.2023', '04.06.2023', '03.06.2023', '28.05.2023', '27.05.2023', '18.05.2023', '30.04.2023', '23.04.2023', '07.04.2023', '18.03.2023', '30.10.2022', '09.10.2022', '30.09.2022', '21.08.2022', '02.07.2022', '04.06.2022', '03.06.2022', '18.04.2022', '06.03.2022', '15.01.2022', '24.10.2021', '17.10.2021', '10.10.2021', '02.10.2021', '21.08.2021', '12.08.2021', '25.07.2021', '10.07.2021', '04.07.2021', '27.06.2021', '26.06.2021', '01.06.2021', '30.05.2021', '08.05.2021', '01.05.2021', '17.10.2020', '20.09.2020', '17.09.2020',

In [46]:
from datetime import datetime
from sklearn.neighbors import KNeighborsRegressor

flight_day_as_mess_date = [int(datetime.strftime(datetime.strptime(d,'%d.%m.%Y'), '%Y%m%d13')) for d in flight_day_list]

df_wind['is_flight_day'] = df_wind.apply(lambda row: 1 if row['MESS_DATUM'] in flight_day_as_mess_date else 0, axis=1)

X, y = df_wind[['strength','wind_alignment','precip']].to_numpy(), df_wind['is_flight_day'].to_numpy() # features and labels
#print(X.shape, y.shape)

# perceptron ?  https://en.wikipedia.org/wiki/Perceptron - no separating hyperplane
# kNN ?
neigh = KNeighborsRegressor(n_neighbors=5, weights= 'distance')
neigh.fit(X, y)

#ytilde = neigh.predict(X)
df_wind['knn'] =  df_wind.apply(lambda row:  neigh.predict([[row['strength'],row['wind_alignment'],row['precip']]]), axis=1)

df_missed = df_wind[(df_wind['knn']>= 0.4) & (df_wind['is_flight_day']==0)].copy()

df_missed['day'] = df_missed.apply(lambda row: datetime.strptime( str(row['MESS_DATUM'])[:8], "%Y%m%d").strftime('%a'), axis=1 )
df_missed.head(20)

,MESS_DATUM,strength,direction,precip,wind_alignment,is_flight_day,knn,day
605525,2019021713,4.4,160,0.0,0.586824,1,[1.0],Sun
605693,2019022413,1.3,200,0.0,0.030154,1,[1.0],Sun
606365,2019032413,2.6,270,0.0,0.750000,1,[1.0],Sun
606485,2019032913,2.5,240,0.0,0.250000,1,[1.0],Fri
606773,2019041013,3.8,70,0.0,0.413176,1,[0.5],Wed
607013,2019042013,2.7,30,0.0,0.000000,1,[1.0],Sat
608381,2019061613,3.5,190,0.0,0.116978,1,[1.0],Sun
608525,2019062213,4.5,80,0.0,0.586824,1,[0.6666666666666666],Sat
609557,2019080413,2.2,240,0.0,0.250000,1,[0.5],Sun
609989,2019082213,4.5,200,0.0,0.030154,1,[1.0],Thu
